In [1]:
import numpy as np
from preprocessing.text import tokenize
from models.llm import load_llm
from models.classifier import load_classifier_distilbert, load_classifier_bert, device
from tqdm import tqdm
from preprocessing.image import preprocess_and_check_image
from database.connect import conn
from preprocessing.document import extract_text_from_pdf
from tqdm import tqdm
import os
import wandb

/home/nightstalker/miniconda3/envs/pytorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
wandb.init(
    # set the wandb project where this run will be logged
    project="generating-impossible-querry-pipeline",
    name="gemini-bert-impossible-query-pipeline",
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mehdinejjar (mehdinejjar-al-akhawayn-university). Use `wandb login --relogin` to force relogin


In [3]:
llm = load_llm()
trainer, tokenizer = load_classifier_bert()

In [4]:
cursor = conn.cursor()
cursor.execute("SELECT * FROM Baby_Bath_Skin_and_Grooming;") 
rows = cursor.fetchall()

In [5]:
IMAGE_INDEX = 4
DOC_INDEX = None
PRIMARY_KEY = ['id']
user_prompt = "Is this item a shampoo with dispenser pump?"
guidelines = """IMPORTANT GUIDELINES:
1. Provide "Match Score" (0–100).
2. EXAMPLE: If shampoo with dispenser pump: match is 100. If it’s a shampoo match is [50-80]. 
3. If totally unrelated, near 0."""

picture_prompt = "analyze the picture"
row_prompt = "analyze the row data"
document_prompt = "analyze the document"

image_flag = 1
document_flag = 0

columns = [col[0] for col in cursor.description]
pk_index = [i for i in range(len(columns)) if columns[i] in PRIMARY_KEY]

columns, pk_index

(['id',
  'name',
  'main_category',
  'sub_category',
  'image',
  'ratings',
  'no_of_ratings',
  'discount_price',
  'actual_price'],
 [0])

In [6]:
results_text = []
PK = []
indices = []
predicted_classes = []

llm_prompt = user_prompt + guidelines

def text_llm(row):
    prompt = f"{row_prompt}.\n{llm_prompt}"
    text_data = " | ".join([f"{columns[j]}: {row[j]}" for j in range(len(columns)) if j != IMAGE_INDEX])
    result = llm.generate_content(
        [f"Row data: {text_data}\n\n", prompt]
    )
    return result.text

def image_llm(image, row):
    prompt = f"{picture_prompt}, and {row_prompt}.\n{llm_prompt}"
    text_data = " | ".join([f"{columns[j]}: {row[j]}" for j in range(len(columns)) if j != IMAGE_INDEX])
    result = llm.generate_content(
        [image, "\n\n", f"Row data: {text_data}\n\n", prompt]
    )
    return result.text

def document_llm(document, row):
    prompt = f"{document_prompt}, and {row_prompt}.\n{llm_prompt}"
    text_data = " | ".join([f"{columns[j]}: {row[j]}" for j in range(len(columns)) if j != DOC_INDEX])
    result = llm.generate_content(
        [document, "\n\n", f"Row data: {text_data}\n\n", prompt]
    )
    return result.text
def image_document_llm(image, document, row):
    prompt = f"{picture_prompt}, {document_prompt}, and {row_prompt}.\n{llm_prompt}"
    text_data = " | ".join([f"{columns[j]}: {row[j]}" for j in range(len(columns)) if j != IMAGE_INDEX and j != DOC_INDEX])
    result = llm.generate_content(
        [image, document, "\n\n", f"Row data: {text_data}\n\n", prompt]
    )
    return result.text


for i in tqdm(range(len(rows))):

    if not image_flag and not document_flag:
        result_text = text_llm(rows[i])
    
    elif image_flag and not document_flag:
        image = preprocess_and_check_image(rows[i][IMAGE_INDEX])
        if image:
            result_text = image_llm(image, rows[i])
        else:
            result_text = text_llm(rows[i])

    elif not image_flag and document_flag:
        ## Apply logic of reading pdf
        document = extract_text_from_pdf(rows[i][DOC_INDEX])
        if document:
            result_text = document_llm(document, rows[i])
        else:
            result_text = text_llm(rows[i])
        
    elif image_flag and document_flag:
        document = extract_text_from_pdf(rows[i][DOC_INDEX])
        image = preprocess_and_check_image(rows[i][IMAGE_INDEX])
        if document and image:
            result_text = image_document_llm(image, document, rows[i])
        elif document and not image:
            result_text = document_llm(document, rows[i])
        elif not document and image:
            result_text = image_llm(image, rows[i])
        elif not document and not image:
            result_text = text_llm(rows[i])

    strip_text = result_text.replace('\n', ' ').replace('*', '')
    
    classifier_text = f"Question: {user_prompt} Answer: {strip_text}"
    results_text.append(classifier_text)
    
    dataset_subset = tokenize(classifier_text, tokenizer)
    predictions = trainer.predict(dataset_subset)
    
    predicted_class = np.argmax(predictions.predictions, axis=1)[0]
    predicted_classes.append(predicted_class)

    PK.append([rows[i][j] for j in pk_index])

  0%|          | 0/302 [00:00<?, ?it/s]

100%|██████████| 302/302 [12:20<00:00,  2.45s/it]


In [7]:
results_text

['Question: Is this item a shampoo with dispenser head? Answer: Match Score: 0  This is a bottle of liquid cleanser specifically designed for cleaning baby bottles, accessories, and vegetables.  While it has a dispensing pump head similar to some shampoos, the labeling clearly indicates its purpose as a cleanser, not a shampoo.  Therefore, it is not a shampoo and certainly not intended for use as one. ',
 'Question: Is this item a shampoo with dispenser head? Answer: Match Score: 0  This is a baby bath seat, not a shampoo or anything related to hair care.  The image and description clearly show a mesh sling designed to support a baby while bathing.  There is no indication of any dispensing mechanism. ',
 'Question: Is this item a shampoo with dispenser head? Answer: Match Score: 0  Analysis:   Product Form: The image and description clearly indicate that this is a massage oil, not a shampoo.  Dispenser: The image shows a flip-top cap, not a pump or dispenser head typically found on sha

In [8]:
predicted_classes

[2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 0,
 1,
 2,
 1,
 1,
 0,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,


In [9]:
rows[:10]

[(11,
  'Luv Lap Liquid Cleanser, Anti-Bacterial, Food Grade, For Baby Bottles, Accessories and Vegetables, 1.5ltr',
  'toys & baby products',
  'Baby Bath, Skin & Grooming',
  'https://m.media-amazon.com/images/I/71p5fCtiCnL._AC_UL320_.jpg',
  Decimal('4.40'),
  '5,883',
  '₹509',
  '₹669'),
 (12,
  'Luv Lap LuvLap Aqua Tales Baby Bather for Baby 0-12 Months, New Born Baby Bath Chair, 3 Position Adjustable, Washable Soft...',
  'toys & baby products',
  'Baby Bath, Skin & Grooming',
  'https://m.media-amazon.com/images/I/91JNkTWv4TL._AC_UL320_.jpg',
  Decimal('4.10'),
  '3,200',
  '₹899',
  '₹1,599'),
 (13,
  'The Moms Co. Natural Baby Massage Oil with 10 Oils - Sesame Oil, Avocado, Organic Almond, Organic Jojoba, Organic Chamomil...',
  'toys & baby products',
  'Baby Bath, Skin & Grooming',
  'https://m.media-amazon.com/images/I/71EQE76D+hL._AC_UL320_.jpg',
  Decimal('4.40'),
  '2,265',
  '₹331',
  '₹474'),
 (14,
  'Chicco Baby Moments Mild Body Wash Refresh, New Advanced Formula Wi

In [18]:
label = 0
elements =[results_text[i] for i in range(len(results_text)) if predicted_classes[i] == label]
elements

['Question: Is this item a shampoo with dispenser head? Answer: The image clearly shows a bottle of Aveeno Baby Wash & Shampoo with a pump dispenser head. The product description also confirms it is a wash and shampoo.  Match score: 100 ',
 'Question: Is this item a shampoo with dispenser head? Answer: Match Score: 100  The image clearly shows a shampoo bottle with a pump dispenser head. The product name in the data confirms it is "Pigeon Baby Conditioning Shampoo."  Therefore, it perfectly matches the criteria of a shampoo with a dispenser head. ',
 'Question: Is this item a shampoo with dispenser head? Answer: Match Score: 100  Analysis:   Image Analysis: The image clearly shows a plastic bottle with a flip-top cap, which is a type of dispenser head commonly used for shampoos and similar liquid products.    Data Analysis: The product name explicitly states "Softsens Baby Shampoo."  The data also lists it under the sub-category "Baby Bath, Skin & Grooming," which further supports the 

In [19]:
len(elements)

12

In [20]:
pk = [[rows[i][j] for j in pk_index] for i in range(len(results_text)) if predicted_classes[i] == label]
pk

[[41],
 [106],
 [111],
 [159],
 [163],
 [211],
 [243],
 [250],
 [264],
 [269],
 [293],
 [304]]

In [21]:
def pk_clause(pk, PRIMARY_KEY):
    return " AND ".join([f"{PRIMARY_KEY[i]} = {pk[i]}" for i in range(len(pk))])

def where_clause(PK, PRIMARY_KEY): 
    return " OR ".join([f"{pk_clause(single, PRIMARY_KEY)}" for single in PK])

In [22]:
user_query = f"SELECT * FROM Baby_Bath_Skin_and_Grooming WHERE {where_clause(pk, PRIMARY_KEY)}"
user_query

'SELECT * FROM Baby_Bath_Skin_and_Grooming WHERE id = 41 OR id = 106 OR id = 111 OR id = 159 OR id = 163 OR id = 211 OR id = 243 OR id = 250 OR id = 264 OR id = 269 OR id = 293 OR id = 304'

In [23]:
cursor.execute(user_query) 
rows_filtered = cursor.fetchall()

In [24]:
rows_filtered

[(41,
  'Aveeno Baby Gentle Wash & Shampoo with Natural Oat Extract, Tear-Free &, Lightly Scented, 18 fl',
  'toys & baby products',
  'Baby Bath, Skin & Grooming',
  'https://m.media-amazon.com/images/I/51Gpjipr8SL._AC_UL320_.jpg',
  Decimal('4.40'),
  '211',
  '₹1,589',
  '₹1,999'),
 (106,
  'Pigeon Baby Conditioning Shampoo, For Newborns, Strengthens and Nourishes Hair, Enriched with Chamomile, Rosehip and Olive...',
  'toys & baby products',
  'Baby Bath, Skin & Grooming',
  'https://m.media-amazon.com/images/I/51D3ndPw0FL._AC_UL320_.jpg',
  Decimal('4.20'),
  '581',
  '₹317',
  '₹435'),
 (111,
  'Softsens Baby TEAR FREE Shampoo | Mild, Hypoallergenic , Paraben & Sufate Free 200ml',
  'toys & baby products',
  'Baby Bath, Skin & Grooming',
  'https://m.media-amazon.com/images/I/51QyP4sokVL._AC_UL320_.jpg',
  Decimal('4.30'),
  '184',
  '₹122',
  '₹175'),
 (159,
  "Mother Sparsh Plant Powered Natural Baby Shampoo With Vanilla Oil, Vitamin E & Coconut Oil | Nourishes Baby's Scalp & H

In [17]:
# save list as file 
with open('results_text.txt', 'w') as f:
    for line in results_text:
        f.write(line + '\n')

np.save('predictions.npy',np.array(predicted_classes))